In [2]:
# IMPORT DEPENDENCIES
# FOR DATA
import pandas as pd
# import os
import csv
import math
import numpy as np
from numpy import logspace
import time
import datetime
# import requests
# import datefinder

# # FOR SQL LITE
# from sqlalchemy import create_engine
# from datetime import date

# # FOR PLOTTING
import matplotlib.pyplot as plt
from matplotlib import style
style.use("fivethirtyeight")
from matplotlib import rcParams
rcParams['figure.figsize'] = 10, 8

# FOR MODELING
from sklearn.model_selection import train_test_split


# FOR KNN AND OTHER MODELS
from scipy.optimize import curve_fit
# from splinter import Browser
# from bs4 import BeautifulSoup as BS
from sklearn.svm import SVC
from sklearn.datasets import load_iris
from sklearn.datasets import make_blobs
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification


In [3]:
file_to_load = "source_files/Fires1.csv"
fire_data = pd.read_csv(file_to_load)
fire_data.head()

C:\Users\amoha\anaconda31\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (8,10,11,12,13,14,15,16,17,18,35,37) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,OBJECTID,FOD_ID,FPA_ID,SOURCE_SYSTEM_TYPE,SOURCE_SYSTEM,NWCG_REPORTING_AGENCY,NWCG_REPORTING_UNIT_ID,NWCG_REPORTING_UNIT_NAME,SOURCE_REPORTING_UNIT,SOURCE_REPORTING_UNIT_NAME,...,FIRE_SIZE_CLASS,LATITUDE,LONGITUDE,OWNER_CODE,OWNER_DESCR,STATE,COUNTY,FIPS_CODE,FIPS_NAME,Shape
0,1,1,FS-1418826,FED,FS-FIRESTAT,FS,USCAPNF,Plumas National Forest,511,Plumas National Forest,...,A,40.036944,-121.005833,5.0,USFS,CA,63,63.0,Plumas,NaN
1,2,2,FS-1418827,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.933056,-120.404444,5.0,USFS,CA,61,61.0,Placer,NaN
2,3,3,FS-1418835,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.984167,-120.735556,13.0,STATE OR PRIVATE,CA,17,17.0,El Dorado,NaN
3,4,4,FS-1418845,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.913333,5.0,USFS,CA,3,3.0,Alpine,NaN
4,5,5,FS-1418847,FED,FS-FIRESTAT,FS,USCAENF,Eldorado National Forest,503,Eldorado National Forest,...,A,38.559167,-119.933056,5.0,USFS,CA,3,3.0,Alpine,NaN


In [11]:
clean_fire_df = fire_data[['FIRE_SIZE','STATE','DISCOVERY_DATE','LATITUDE','LONGITUDE']].copy()
clean_fire_df = clean_fire_df.reset_index(drop=True)
clean_fire_df.shape

(1880465, 5)

In [12]:
clean_fire_df['DISCOVERY_DATE'] = pd.to_datetime(clean_fire_df['DISCOVERY_DATE'], format='%Y%m')
clean_fire_df.head()

ValueError: unconverted data remains: 03

In [13]:
clean_fire_df.dtypes

FIRE_SIZE         float64
STATE              object
DISCOVERY_DATE    float64
LATITUDE          float64
LONGITUDE         float64
dtype: object

In [14]:
clean_fire_df.head()

,FIRE_SIZE,STATE,DISCOVERY_DATE,LATITUDE,LONGITUDE
0,0.10,CA,2453403.5,40.036944,-121.005833
1,0.25,CA,2453137.5,38.933056,-120.404444
2,0.10,CA,2453156.5,38.984167,-120.735556
3,0.10,CA,2453184.5,38.559167,-119.913333
4,0.10,CA,2453184.5,38.559167,-119.933056


In [16]:
fire_count = clean_fire_df.groupby(['STATE']).count()
fire_count.rename(columns={'FIRE_SIZE':'total_acres_burned'}, inplace=True)
fire_count = fire_count.reset_index()
fire_count.head()

fire_count.iloc[:,1:2].shape
fire_count.head()


,STATE,total_acres_burned,DISCOVERY_DATE,LATITUDE,LONGITUDE
0,AK,12843,12843,12843,12843
1,AL,66570,66570,66570,66570
2,AR,31663,31663,31663,31663
3,AZ,71586,71586,71586,71586
4,CA,189550,189550,189550,189550


In [ ]:
def build_and_compile_model(norm):
  model = keras.Sequential([
      norm,
      layers.Dense(64, activation='relu'),
      layers.Dense(64, activation='relu'),
      layers.Dense(1)
  ])

  model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
  return model

In [ ]:
train_features = fire_count.iloc[:,1:2].sample(frac=0.8, random_state=0)
test_features = fire_count.iloc[:,1:2].drop(train_features.index)
test_features.head()

sample_train = train_features.copy()
sample_test = test_features.copy()
train_labels = sample_train.pop('fire_Count')
test_labels = sample_test.pop('fire_Count')

train_features.shape

In [ ]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(train_labels))

In [ ]:
dnn_model = build_and_compile_model(normalizer)
dnn_model.summary()

In [ ]:
%%time
history = dnn_model.fit(
    train_features, train_labels,
    validation_data=(test_features, test_labels),
    verbose=0, epochs=1500)

In [ ]:
test_predictions = dnn_model.predict(test_features).flatten()
print(test_predictions.shape)
test_labels.shape

In [ ]:
plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [Acres Burned]')
plt.ylabel('Predictions [Acres Burned]')

lims = [0, 600]
plt.xlim(lims)
plt.ylim(lims)
mpl.style.use('seaborn')
plt.title('US Predicted vs Actual')
_ = plt.plot(lims, lims, 'C2')
plt.savefig('data_sets/conus_predicted_acres_burned.png')
plt.show()